In [ ]:
#ENSEMBLE TEST 2
##NN Ensemble 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
import gc
import time
import sys
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler

from keras import models
from keras import layers
from keras import optimizers

from sklearn.utils import shuffle

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(3,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

print(model.summary())

N_FOLDS = 5.0

kf = KFold(n_splits=int(N_FOLDS), random_state=2019, shuffle=True)



In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")


In [ ]:
#Load previous data

catb_df = pd.read_csv("../input/catboost_output.csv")
lgbm_df = pd.read_csv("../input/output_test_lgbm_base3.csv")
nn_df = pd.read_csv("../input/full_nn_equal_output.csv")

submission_catb_df = pd.read_csv("submission_catboost.csv")
submission_lgbm_df = pd.read_csv("submission_lgbm_base3.csv")
submission_full_nn_df = pd.read_csv("submission_full_nn_equal.csv")

combined_df = pd.DataFrame({"ID_code": catb_df.ID_code.values})
combined_df['cat_out'] = catb_df['output']
combined_df['lgbm_out'] = lgbm_df['output']
combined_df['full_nn_out'] = nn_df['output']
combined_df['target1'] = train_df['target']
combined_df['target0'] = 1 - train_df['target']

#drop 90% of 0 scores to make balanced input for NN
#combined_df = combined_df.drop(combined_df[combined_df['target0'] > 0.9].sample(frac=0.1).index)
zero_subset_df = combined_df[combined_df['target0'] > 0.9].sample(frac = 0.1)
ones_subset_df = combined_df[combined_df['target1'] > 0.9]

frames = [zero_subset_df, ones_subset_df]

combined_df = pd.concat(frames)
combined_df = shuffle(combined_df)

combined_test_df = pd.DataFrame({"ID_code": test_df.ID_code.values})
combined_test_df['cat_out'] = submission_catb_df['target']
combined_test_df['lgbm_out'] = submission_lgbm_df['target']
combined_test_df['full_nn_out'] = submission_full_nn_df['target']

zero_subset_df.to_csv('../input/zeros_sample.csv')
ones_subset_df.to_csv('../input/ones_sample.csv')
combined_df.to_csv('../input/combined.csv')

#idx = [c for c in combined_df.columns if c not in ['ID_code', 'target']]
features = [c for c in combined_df.columns if c not in ['ID_code', 'target', 'target1', 'target0']]
target = combined_df.loc[:, ['target1', 'target0']]

y_test_pred = np.zeros(len(test_df))
    
for idx, (train_index, valid_index) in enumerate(kf.split(combined_df)):
    print("Fold {}".format(idx))
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    X_train, X_valid = combined_df[features].iloc[train_index,:], combined_df[features].iloc[valid_index,:]
    model.fit(X_train, y_train, epochs=20, batch_size = 512, validation_data =(X_valid, y_valid))
    pred = model.predict_proba(X_valid)
    print(pred.shape)
    print( "  auc = ", roc_auc_score(y_valid, pred) )
    y_test_pred += model.predict_proba(combined_test_df[features])[:,0]
    print(y_test_pred.shape)

#save base submission
sub_df1 = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df1["target"] = y_test_pred / N_FOLDS
sub_df1.to_csv("submission_nn_ensemble2.csv", index=False)
#make combined data set for comparison
combined_test_df['nn_out'] = y_test_pred / N_FOLDS
combined_test_df.to_csv("../input/combined_output_ensemble2.csv")